In [1]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

print(keras.__version__)

2022-05-27 22:44:01.153285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-27 22:44:01.153309: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1
2.9.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [4]:
# Wczytaj dane treningowe i testowe
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

train_set = pd.read_csv('data/adult/adult.data', sep=", ",header = None)
test_set = pd.read_csv('data/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set

col_labels = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 
              'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
             'wage_class']
train_set.columns = col_labels
test_set.columns = col_labels

train = train_set.replace('?', np.nan).dropna()
test = test_set.replace('?', np.nan).dropna()

train_set.head()

/tmp/ipykernel_3968490/3249197133.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  train_set = pd.read_csv('data/adult/adult.data', sep=", ",header = None)
/tmp/ipykernel_3968490/3249197133.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  test_set = pd.read_csv('data/adult/adult.test', sep=", ",skiprows = 1, header = None) # Make sure to skip a row for the test set


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,wage_class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Zad.
Zróbmy szybki preprocesing

In [5]:
dataset = pd.concat([train,test])

dataset['wage_class'] = dataset.wage_class.replace({'<=50K.': 0,'<=50K':0, '>50K.':1, '>50K':1})

dataset.drop(["fnlwgt"],axis=1,inplace=True)

dataset.drop(["education"],axis=1,inplace=True)

x = dataset.groupby('native_country')["wage_class"].mean()

d = dict(pd.cut(x[x.index!=" United-States"],5,labels=range(5)))

dataset['native_country'] = dataset['native_country'].replace(d)

dataset = pd.get_dummies(dataset,drop_first=True)

train = dataset.iloc[:train.shape[0]]
test = dataset.iloc[train.shape[0]:]

X_train = train.drop("wage_class",axis=1)
y_train = train.wage_class

X_test = test.drop("wage_class",axis=1)
y_test = test.wage_class

# TensorBoard

In [6]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [7]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
run_logdir

'./my_logs/run_2022_05_27-22_44_17'

In [7]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

from keras.callbacks import History

history = History()
model = Sequential()
model.add(Dense(100,activation="sigmoid",input_shape=(X_train.shape[1],)))
model.add(Dense(50,activation="sigmoid"))
model.add(Dense(10,activation="sigmoid"))
model.add(Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 100)               4200      
                                                                 
 dense_8 (Dense)             (None, 50)                5050      
                                                                 
 dense_9 (Dense)             (None, 10)                510       
                                                                 
 dense_10 (Dense)            (None, 1)                 11        
                                                                 
Total params: 9,771
Trainable params: 9,771
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(loss="binary_crossentropy",optimizer="Adam", metrics=["accuracy"])

In [10]:
from keras.callbacks import TensorBoard 

tensorboard_cb = TensorBoard(run_logdir)
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test), callbacks=[tensorboard_cb])

Epoch 1/30
943/943 [==============================] - 5s 4ms/step - loss: 0.4660 - accuracy: 0.7815 - val_loss: 0.4067 - val_accuracy: 0.8016
Epoch 2/30
943/943 [==============================] - 4s 4ms/step - loss: 0.3826 - accuracy: 0.8125 - val_loss: 0.3707 - val_accuracy: 0.8275
Epoch 3/30
943/943 [==============================] - 4s 4ms/step - loss: 0.3747 - accuracy: 0.8163 - val_loss: 0.3711 - val_accuracy: 0.8167
Epoch 4/30
943/943 [==============================] - 4s 4ms/step - loss: 0.3719 - accuracy: 0.8158 - val_loss: 0.3980 - val_accuracy: 0.8060
Epoch 5/30
943/943 [==============================] - 4s 4ms/step - loss: 0.3709 - accuracy: 0.8156 - val_loss: 0.3689 - val_accuracy: 0.8155
Epoch 6/30
943/943 [==============================] - 4s 4ms/step - loss: 0.3746 - accuracy: 0.8107 - val_loss: 0.3742 - val_accuracy: 0.8100
Epoch 7/30
943/943 [==============================] - 4s 4ms/step - loss: 0.3740 - accuracy: 0.8136 - val_loss: 0.3689 - val_accuracy: 0.8189
Epoch 

o start the TensorBoard server, one option is to open a terminal, if needed activate the virtualenv where you installed TensorBoard, go to this notebook's directory, then type:

```python
$ tensorboard --logdir=./my_logs --port=6006
```

You can then open your web browser to localhost:6006 and use TensorBoard. Once you are done, press Ctrl-C in the terminal window, this will shutdown the TensorBoard server.

Alternatively, you can load TensorBoard's Jupyter extension and run it like this:

```python
%load_ext tensorboard
%tensorboard --logdir=./my_logs --port=6006
```

Gdyby nie działał tensorboard

https://github.com/pytorch/pytorch/issues/22676
```python
pip uninstall tb-nightly tensorboardX tensorboard
pip install tensorboard
```

# Zad

* Naucz dowolny model na zbiorze MNIST.
* Wyświetl krzywe uczenia w tensorboard

In [14]:
import tensorflow_datasets as tfds
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\slabj\tensorflow_datasets\mnist\3.0.1.incomplete01S4Z0\mnist-train.tfrecord*...:   0%|     …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\slabj\tensorflow_datasets\mnist\3.0.1.incomplete01S4Z0\mnist-test.tfrecord*...:   0%|      …

Dataset mnist downloaded and prepared to C:\Users\slabj\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [15]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [16]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [54]:
model2 = Sequential()
history2 = History()

In [55]:
model2.add(Flatten(input_shape=(28, 28)))
model2.add(Dense(128,activation="relu"))
model2.add(Dense(10,activation="sigmoid"))

In [56]:
model2.compile(loss="SparseCategoricalCrossentropy",optimizer="Adam", metrics=["accuracy"])

In [57]:
run_logdir = get_run_logdir()
run_logdir

'.\\my_logs\\run_2022_05_25-16_42_04'

In [58]:
tensorboard_cb2 = TensorBoard(run_logdir)
history2 = model2.fit(ds_train, epochs=6, validation_data=ds_test, callbacks=[tensorboard_cb2])

Epoch 1/6
469/469 [==============================] - 3s 4ms/step - loss: 0.3590 - accuracy: 0.9022 - val_loss: 0.1925 - val_accuracy: 0.9448
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1663 - accuracy: 0.9529 - val_loss: 0.1400 - val_accuracy: 0.9595
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1184 - accuracy: 0.9661 - val_loss: 0.1154 - val_accuracy: 0.9649
Epoch 4/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0914 - accuracy: 0.9738 - val_loss: 0.1020 - val_accuracy: 0.9688
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0749 - accuracy: 0.9782 - val_loss: 0.0937 - val_accuracy: 0.9726
Epoch 6/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0606 - accuracy: 0.9825 - val_loss: 0.0865 - val_accuracy: 0.9753
